In [12]:
import numpy as np
from osgeo import gdal,ogr,osr
import matplotlib.pyplot as plt
import os, sys

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,10) # Make the figures a bit bigger

def GetExtent(gt,cols,rows):
    ''' Return list of corner coordinates from a geotransform

        @type gt:   C{tuple/list}
        @param gt: geotransform
        @type cols:   C{int}
        @param cols: number of columns in the dataset
        @type rows:   C{int}
        @param rows: number of rows in the dataset
        @rtype:    C{[float,...,float]}
        @return:   coordinates of each corner
    '''
    ext=[]
    xarr=[0,cols]
    yarr=[0,rows]

    for px in xarr:
        for py in yarr:
            x=gt[0]+(px*gt[1])+(py*gt[2])
            y=gt[3]+(px*gt[4])+(py*gt[5])
            ext.append([x,y])
            #print x,y
        yarr.reverse()
    return ext

def ReprojectCoords(coords,src_srs,tgt_srs):
    ''' Reproject a list of x,y coordinates.

        @type geom:     C{tuple/list}
        @param geom:    List of [[x,y],...[x,y]] coordinates
        @type src_srs:  C{osr.SpatialReference}
        @param src_srs: OSR SpatialReference object
        @type tgt_srs:  C{osr.SpatialReference}
        @param tgt_srs: OSR SpatialReference object
        @rtype:         C{tuple/list}
        @return:        List of transformed [[x,y],...[x,y]] coordinates
    '''
    trans_coords=[]
    transform = osr.CoordinateTransformation( src_srs, tgt_srs)
    for x,y in coords:
        x,y,z = transform.TransformPoint(x,y)
        trans_coords.append([x,y])
    return trans_coords

dpath = r'/home/mirabot/googledrive/deeplearning/seaiceprj/G02159GeoTiffs_3c/'
cpath = r'/home/mirabot/googledrive/deeplearning/seaiceprj/G02159GeoTiffs_3c_centroids500m/'
gt_list = os.listdir(dpath)

for raster in gt_list:
    ds=gdal.Open(dpath+raster)
    print raster
    gt=ds.GetGeoTransform()
    cols = ds.RasterXSize
    rows = ds.RasterYSize
    ext=GetExtent(gt,cols,rows)

    src_srs=osr.SpatialReference()
    src_srs.ImportFromWkt(ds.GetProjection())
    #tgt_srs=osr.SpatialReference()
    #tgt_srs.ImportFromEPSG(4326)
    tgt_srs = src_srs.CloneGeogCS()

    geo_ext=ReprojectCoords(ext,src_srs,tgt_srs)

    img = np.array(ds.GetRasterBand(1).ReadAsArray())
    #imgplot = plt.imshow(img)
    ds = None

    ymax = ext[0][1]
    ymin = ext[1][1]
    xmax = ext[2][0]
    xmin = ext[0][0]
    print ymax,ymin,xmax,xmin

    rg = 10000
    cell_size = 500
    cell_num = rg/cell_size
    cell_centroids = np.zeros((cell_num,cell_num,2))

    out = open(cpath+raster[:-4]+'_500m_centroids.csv', 'w')

    for row in range(0, cell_num):
        for col in range(0, cell_num):
            cell_centroids[row,col,1] = ymax - (2*(row+1)-1)*cell_size/2
            cell_centroids[row,col,0] = xmin + (2*(col+1)-1)*cell_size/2

            out.write('%d,%d,%f,%f' % (row,col,cell_centroids[row,col,0],cell_centroids[row,col,1]))
            out.write('\n')
    out.close()


esiber_2000jul13b_3c.tif
9118643.32862 9108643.32862 459676.30482 449676.30482
cafram_2000jun27a_3c.tif
9452138.97877 9442138.97877 477725.352962 467725.352962
cacana_2000jun28a_3c.tif
9433739.98 9423739.98 482111.991 472111.991
esiber_2000jul04a_3c.tif
9101657.91437 9091657.91437 556443.660683 546443.660683
cacana_2000jun22a_3c.tif
9438368.55 9428368.55 485707.795 475707.795
beaufo_2000jul21b_3c.tif
8102758.36 8092758.36 408947.665 398947.665
cacana_2000jun25a_3c.tif
9443817.24 9433817.24 474044.095 464044.095
esiber_2000aug13a_3c.tif
9116330.17063 9106330.17063 550655.930618 540655.930618
cacana_2000aug15a_3c.tif
9437509.71 9427509.71 487318.108 477318.108
cacana_2000jul27a_3c.tif
9444840.92805 9434840.92805 533603.326272 523603.326272
beaufo_2000sep02a_3c.tif
8096791.82 8086791.82 595437.267 585437.267
beaufo_2000jul14a_3c.tif
8109702.47 8099702.47 416589.085 406589.085
esiber_2000jul28a_3c.tif
9114276.14431 9104276.14431 466860.375662 456860.375662
beaufo_2000aug25c_3c.tif
8099716.